# **TP Block Chain : Interaction avec le smart contract sans utiliser Web3**
# **KARTOBI Sofiane**


## Objectif : setter la valeur de greeting à travers la fonction setGreeting() sans faire à recours à Web3

### Plan d'action : 
Afin de réaliser ce TP, nous devons passer par plusieurs étapes clées :
- Tout d'abord on récupére le node RPC et on s'y connecte 
- Ensuite, on load la clé privée du compte MetaMask (jusqu'à présent rien de nouveau par rapport au devoir maison)
- Puis, on doit construire la fonction qui va créer la transaction qui nous intéresse (faire appel à setGreeting en passant la nouvelle valeur en paramétre), la faire signé et enfin l'envoyer.
- Afin d'envoyer la transaction, la fonction qu'on construira aura besoin de divers paramétres qu'on doit fixer, à savoir 
    - La data de la transaction (bytecode, aka le goulot de toute cette manipulation)
    - Le nonce et gas price qu'on doit récupérer à travers des requêtes http en utilisant request et payload 
    - Le reste demeure similaire (adresse du compte from, gas limit, etc...)
    

### Etape 0 : Préparation et importation

In [1]:
import requests
import hashlib
import json

In [2]:
contract_address="0x80e70d63EeEBdF2ea4EfA53Ed84CD86398A14730"
wallet_address="0xada2E183e53E1DDe87D4b40f1f4C2877481a1Ca2"

In [3]:
contract_abi = '''
    [
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_greeting",
				"type": "string"
			}
		],
		"name": "setGreeting",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [],
		"name": "greeting",
		"outputs": [
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]
'''

- Importation de la clé privée

In [4]:
with open("MetaMaskPK.txt", "r") as f:
    private_key = f.read()

### Etape 1 : Création de la fonction get_nonce() et get_gasPrice()

- On définit le node rpc de la chainID 97
- On fixe la méthode eth_getTransactionCount qui nous permettra d'avoir la valeur du nonce et eth_gasPrice qui nous aidera à avoir le prix du gaz actuellement
- Ici je me suis aidé de Bard pour définir ces fonctions étant donné que je ne suis pas très familiarisé avec la librairie request 

In [5]:
rpc_node = "https://data-seed-prebsc-1-s1.binance.org:8545/"
nonce_method = "eth_getTransactionCount"
gasPrice_method ="eth_gasPrice"

#### Récupération du nonce 

In [6]:
def get_nonce(wallet_address): 
  

  # Preparing the JSON-RPC payload
  payload = {
    "jsonrpc": "2.0",
    "method": nonce_method,
    "params": [wallet_address, "latest"],
    "id": 1
  }

  # Send the request and extract the gas price
  response = requests.post(rpc_node, json=payload)
  data = response.json()

  if response.status_code == 200:
    nonce = int(response.json()['result'], 16) 
    return nonce
  else:
    print("Erreur lors de la récupération du nonce")

- On vérifie si elle est bien implémentée

In [7]:
Nonce = get_nonce(wallet_address=wallet_address)
Nonce

11

#### Récupération du gas Price :

In [8]:
def get_gas_price():
  
  # Prepare the JSON-RPC payload
  payload = {
      "jsonrpc": "2.0",
      "method": gasPrice_method,
      "params": [],
      "id": 1
  }

  # Send the request and extract the gas price
  response = requests.post(rpc_node, json=payload)
  data = response.json()

  if response.status_code == 200:
    gas_price = int(data["result"], 16)
    return gas_price
  else:
    raise Exception(f"Error getting gas price: {data['error']['message']}")


- Vérification du prix 

In [9]:
gas_price = get_gas_price()

print(f"Gas Price: {gas_price} wei")

Gas Price: 5000000000 wei


### Etape 2 : Conception du bytecode (data de la transaction)

- Afin de mieux comprendre la structure du Bytecode de la data de la transaction, j'ai lancé un appel test de la fonction setGreetin() en utilisant Remix avec la valeur "Sofiane", par la suite j'ai visualisé  et analysé sur bscscan sa structure pour pouvoir comprendre comment concevoir le bytecode

![Image](Capture.png)

- Voici sa structure : 

![Image](Capture2.png)

Input Data contient 4 parties : 
- MethodID : qui est le hash de la signature (prototype) de la fonction avec le keccak en hexadecimal
- [0] : 64 digits (32 byte) comportant un offset et finissant par 20 (indiquant la taille du prochain input data element en hex donc 32 byte)
- [1] : 32 byte avec un offset et à la fin 7 (ce dernier représente la taille de la chaine de chractere en hexadecimal) 
- [2] : 32 byte comportant au début la valeur de la chaine de charactére qui est encodée en hexadecimal et le reste est comblé avec des 0

- Vérifions le hash de la signature de setGreeting(string) 

In [10]:
from Crypto.Hash import keccak

# la signature de la fonction à hacher
string_to_hash = "setGreeting(string)"

# Création de l'objet de hachage Keccak-256
function_signature = keccak.new(digest_bits=256, data=string_to_hash.encode("utf-8")).hexdigest()[:8]
print(f"la signature de la fonction : {function_signature}")

la signature de la fonction : a4136862


- Nice, ça a l'air d'être bon !
- Vérifions le reste, la taille de "Sofiane" ainsi que son encodage en hexadecimal :

In [11]:
string = "Sofiane"
print(string.encode("utf-8").hex())
print(hex(len(string)))

536f6669616e65
0x7


#### Perfect !

- A présent, nous pouvons simuler cela pour envoyer notre transaction avec une nouvelle valeur "Porsche"

In [12]:
new_value = "Porsche"

- Le hashage de la signature et [0] ne vont pas changer 
- Tout ce qui changera est au niveau de [1] qui aura la nouvelle taille et [2] la nouvelle valeur encodé en hexadécimal

#### Création du bytecode

In [13]:
def create_greeting_bytecode(greeting):
 
  next_input_size = int.to_bytes(32, 32, byteorder="big").hex()
  greeting_length = int.to_bytes(len(greeting.encode("utf-8")), 32, byteorder="big").hex()
  greeting_hex = greeting.encode("utf-8").hex()
  while len(greeting_hex) < 64: 
    greeting_hex += '0'
    
  # Create the bytecode
  bytecode = f"0x{function_signature}{next_input_size}{greeting_length}{greeting_hex}"
  return bytecode

- Let's check...

In [14]:
bytecode = create_greeting_bytecode(new_value)
print(f"Bytecode: {bytecode}")

Bytecode: 0xa413686200000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000007506f727363686500000000000000000000000000000000000000000000000000


In [15]:
len(bytecode)

202

### Etape 3 : Création et envoi de la transaction

- On aura besoin d'installer eth-account pour se faire

In [16]:
!pip install eth-account


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import eth_account

In [18]:
contract_address

'0x80e70d63EeEBdF2ea4EfA53Ed84CD86398A14730'

- Conception de la transaction en utilisant les précédentes fonctions get_nonce() et get_gas_price()
- On passe en argument le bytecode qu'on a conçu
- Signature avec la clé privée
- Convertion en hexadecimal

In [19]:
transaction = {
    'nonce': get_nonce(wallet_address=wallet_address),
    'gasPrice': get_gas_price(),
    'gas': 100000,
    'to': contract_address,
    'data': bytecode,
}


signed_transaction = eth_account.Account.sign_transaction(transaction, private_key=private_key)


raw_transaction_hex = signed_transaction.rawTransaction.hex()

- Ici on envoie la transaction signée en hex avec le même protocole à l'aide payload et request et à la fin on recoit le hash de la transaction

In [20]:
headers = {'Content-Type': 'application/json'}
payload = {
    'jsonrpc': '2.0',
    'method': 'eth_sendRawTransaction',
    'params': [raw_transaction_hex],
    'id': 1,
}

response = requests.post(rpc_node, data=json.dumps(payload), headers=headers)
transaction_hash = response.json()['result']
print(f"Transaction hash: {transaction_hash}")

Transaction hash: 0xb3da6066813c3d5087eccaf4a26e4243f63ebf821954eb8c16128606a78be353


In [21]:
response.json()

{'jsonrpc': '2.0',
 'id': 1,
 'result': '0xb3da6066813c3d5087eccaf4a26e4243f63ebf821954eb8c16128606a78be353'}

### Vérification en appelant greeting 
- On va appliquer le même principe pour vérifier la valeur de la variable, créer un nouveau bytecode spécifique à l'appel de la nouvelle fonction (qui cette fois n'a pas de paramètre)

In [22]:
function_signature = "greeting()"


function_signature_hash = keccak.new(digest_bits=256, data=function_signature.encode("utf-8")).hexdigest()[:8]


bytecode = "0x" + function_signature_hash 

# requête JSON-RPC 
payload = {
    "jsonrpc": "2.0",
    "method": "eth_call",
    "params": [{
        "to": contract_address,
        "data": bytecode
    }, "latest"],
    "id": 1
}

# Envoi de la requête
response = requests.post(rpc_node, json=payload)


if response.status_code == 200:
    r = response.json()['result']
    greeting_value = bytes.fromhex(r[2:]).decode('utf-8')  # Convertir la réponse hexadécimale en chaîne de caractères
    print(f"Valeur actuelle de 'greeting': {greeting_value}")
else:
    print("Erreur lors de la récupération de la valeur de 'greeting'")


Valeur actuelle de 'greeting':                                                                Porsche                         


- It's time to check si la valeur a bien changé...
- Sur Remix :

![Image](Capture3.png)

- Sur bscscan, je check input data puis je clique sur "Decode input data" et on obtient : 

![Image](Capture6.png)
![Image](Capture7.png)


#### **IT FINALLY WORKED !** 

and yes I like Porsche ;)

## **Problémes rencontrés :**
- Après avoir tout implémenté, je me suis rendu sur bsc scan pour vérifier l'état de la transaction mais cette dernière ne passer pas, j'ai donc augmenter gas limit, puis j'ai vérifier mes bnb sur metamask, mais rien ne voulait changer. J'avais toujours cette keyError : 'result' ce qui signifiait que dans le json obtenu je n'avais pas de résultat et que donc il fallait creuser davantage.
J'ai demandé à chatgpt, j'ai parcouru les forums sur stack exchange et reddit mais aucune piste n'était intéressante.
- Sur Bsc Scan l'erreur semblait être liée à l'implementation du smart contract, hors il est basique et tout le monde avait le même, j'ai vraiement eu du mal à m'en sortir

![Image](Capture5.png)

![Image](Capture4.png)


- Finalement, je me suis rendu compte que j'avais mal implémnté le bytecode et sa longeur totale était de  216 alors qu'il fallait qu'il soit sur 202, j'ai corrigé cela et la transaction est passé (après presque 2 heures de reflexion intense en recheckant ligne par ligne)
- **Remarque :** en restrant puis run All une nouvelle transaction s'est crée mais la capture date de la 1ére réelle transaction passée